## 从零搭建智能风控模型体系  

源代码来自：https://gitee.com/fadgabadfaf/practice_of_intelligent_risk_control  
项目运行时选择【**风控专用镜像**】  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


### 已安装本实验要用到的库  

**sklearn库**：全称 Scikit-learn，涵盖了分类、回归、聚类、降维、模型选择、数据预处理六大模块，降低机器学习实践门槛，将复杂的数学计算集成为简单的函数，并提供了众多公开数据集和学习案例。官方文档：https://scikit-learn.org  

**scipy库**：Scipy是一个用于数学、科学、工程领域的常用软件包，可以处理插值、积分、优化、快速傅里叶变换、图像处理、常微分方程数值解的求解、信号处理等问题，提供了用于操作和可视化数据的数学算法和便利函数。  

**statsmodels库**：statsmodels是一个包含统计模型、统计测试和统计数据挖掘的常用软件包，对每一个模型都会生成一个对应的统计结果。  

**scorecardpy库**：信贷建模时常用的轻量级Python库，其中内置了德国信用卡数据集。  

**toad库**：专为风险评分卡建模而开发的工具包，封装了数据探索、特征筛选、特征分箱、WOE变换、模型评估、分数转换等内容，极大简化了评分卡建模的复杂程度。  

**category-encoders库**：用于类别数据编码（转换为数值数据）的python库，包含各种编码方法的集合，可以根据需要将其应用于各种类别型数据的特征工程。  

**bayesian_optimization库**：贝叶斯优化实现网络的超参数搜索。  

**xgboost与lightgbm库**：基于 Boosting 框架的主流集成算法。  


## 1. 数据探索EDA  

**数据准备：**获取scorecardpy库中内置的德国信用卡数据集，添加month列数据，该数据集共有1000条数据。  
**通过数据探索了解数据的特征分布，确认数据质量。**

In [1]:
import toad
from utils import data_utils

# 加载添加month列的数据集
german_credit_data = data_utils.get_data()

# 数据探索是指对特征进行统计分析，统计每个特征的缺失率、唯一值个数、最大值、最小值、平均值和趋势性变化等指标
detect_res = toad.detector.detect(german_credit_data)

# 打印前5行, 前4列
#print("前5行, 前4列:")
#print(detect_res.iloc[:5, :4])

# 打印前5行, 第5至9列
#print("前5行, 第5至9列:")
#print(detect_res.iloc[:5, 4:9])

# 打印前5行, 第10至14列
#print("前5行, 第10至14列:")
#print(detect_res.iloc[:5, 9:])

In [2]:
def missrate_by_month(x_with_month, month_col, x_cols):
    """
    按月统计缺失率
    :param x_cols: x变量列名
    :param month_col: 月份时间列名
    :param x_with_month: 包含月份的数据
    :return:
    """
    df = x_with_month.groupby(month_col)[x_cols].apply(lambda x: x.isna().sum() / len(x))
    df = df.T
    df['miss_rate_std'] = df.std(axis=1)
    return df

In [3]:
miss_rate_by_month = missrate_by_month(german_credit_data, month_col='month', x_cols=data_utils.numeric_cols)
print("按月统计缺失率结果: \n", miss_rate_by_month)

按月统计缺失率结果: 
 month                                               2020-01  2020-02  2020-03  \
duration_in_month                                       0.0      0.0      0.0   
credit_amount                                           0.0      0.0      0.0   
age_in_years                                            0.0      0.0      0.0   
present_residence_since                                 0.0      0.0      0.0   
number_of_existing_credits_at_this_bank                 0.0      0.0      0.0   
installment_rate_in_percentage_of_disposable_in...      0.0      0.0      0.0   
number_of_people_being_liable_to_provide_mainte...      0.0      0.0      0.0   

month                                               2020-04  2020-05  \
duration_in_month                                       0.0      0.0   
credit_amount                                           0.0      0.0   
age_in_years                                            0.0      0.0   
present_residence_since                           

## 2. 数据集划分  
将2020年5月份样本作为OOT样本，将1～4月份样本作为训练样本和验证样本，并按照7:3的比例划分训练集和验证集

In [4]:
from sklearn.model_selection import train_test_split

# 选取OOT样本  
oot_set = german_credit_data[german_credit_data['month'] == '2020-05']
# 划分训练集和测试集
train_valid_set = german_credit_data[german_credit_data['month'] != '2020-05']
X = train_valid_set[data_utils.x_cols]
Y = train_valid_set['creditability']
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, random_state=88)
german_credit_data.loc[oot_set.index, 'sample_set'] = 'oot'
german_credit_data.loc[X_train.index, 'sample_set'] = 'train'
german_credit_data.loc[X_valid.index, 'sample_set'] = 'valid'

## 3. 数据预处理  

### 通过EDA了解了数据特征的基本情况，排除了一部分潜在问题。在进行特征选择和建模之前，需要对数据进行预处理，使得数据能够全面地反映全体样本信息，以适用于机器学习模型。  

### 1）异常值检测  
#### a. 基于统计的方法，构建一个概率分布模型，并计算对象符合该模型的概率，把具有低概率的对象视为异常点。例如z-socre检测方法。  
#### b. 基于聚类的方法，采用聚类算法（如K-means、DBSCAN等）将训练样本分成若干类，如果某一个类中的样本非常少，而且类中心和其他所有类的距离都很远，那么这个类中的样本极有可能是异常特征样本。  
#### c. 异常点检测算法，孤立森林（Isolation Forest）是一种应用广泛的异常点检测算法，离群点可以理解为分布稀疏且与密度高的群体较远的点。在特征空间中，分布稀疏的区域表示事件发生在该区域的概率很低，因而可以认为落在这些区域里的数据是异常的。  

### 2）异常值处理  
#### a. 直接删除包含异常特征值的样本。  
#### b. 结合特征含义选择置空异常值，或者填充为其他值。  
#### c. 在风控业务中，检测出的异常值可能是某种欺诈行为产生的，直接抛弃是不妥的，需要深入分析，修复风控体系中的漏洞。  

### 3）缺失值处理  
#### 在实际业务中，非正常缺失和正常缺失往往同时存在。例如，在某段时间，数据接口出现异常，导致这段时间内的特征缺失值，需要首先考虑将这部分样本剔除。如果这部分样本的量较大，那么不宜直接剔除，可以考虑将异常时间段的特征标记为特定值，再根据建模算法确定其他时间段正常缺失的特征值的填充方案。  

### 4）特征无量纲化  
#### 通过特征的标准化将特征值“归一化”到同一量纲。  

### 5）连续特征离散化  
#### 连续特征离散化，也称为特征分箱，是指将连续属性的特征进行分段，使其变成一个个离散的区间。在使用线性模型建立风险评分卡模型时，通常会对连续特征进行离散化分箱。离散化后的特征对异常值会有很强的鲁棒性，降低了线性模型过拟合的风险，模型会更稳定。而在使用XGBoost、LightGBM等算法时，通常不需要分箱。  
#### 单个特征离散化为多个分箱后，再对分箱进行数值转换（如WOE变换），这个过程实际上可以对原本非线性的关系进行线性转化，提高线性模型的表达能力。  
#### 常用的分箱方法有等频分箱、等距分箱、卡方分箱和决策树分箱。  

### 6）类别特征数值化  
#### 有些算法（如LR、SVM等）要求所有特征是数值型变量，但在实际业务中会遇到很多类别型变量，比如性别、学历和职业等。类别型变量分为两种，一种是没有任何先后顺序或等级关系的标称类别型变量（nominal category variable），如性别、省份等；另一种是有先后顺序或等级关系的有序类别型变量（ordinal category variable），如学历、满意程度等。  
#### 处理类别特征时通常采用两种编码方式：一种是无监督编码方式，如有序数编码和one-hot编码；另一种是有监督编码方式，如目标编码和WOE编码。当特征类别取值较多时，通常先进行分箱（常用决策树分箱），合并一些类别后，再对分箱进行编码处理。  

### 7）特征交叉组合  
#### 特征交叉组合可以增加特征的维度，组合的特征能够反映更多的非线性关系。在实践中，通常会对类别特征进行组合，对于连续特征可以先进行离散化分箱，再进行组合，也可以直接进行特征交叉衍生。  
#### 特征交叉衍生的常用方法包括：利用数值型特征之间的加减乘除操作得到新特征；对已选定特征进行奇异值分解（SVD），将奇异值作为新特征；根据已选特征进行聚类，将所有在类别的平均目标值或出现最多的值作为新特征，或者将所有类别与其他类别的距离作为新特征等。也可以利用深度学习技术衍生新特征，例如将神经网络中某层的值作为新特征。  
#### 不难发现，特征交叉组合会导致特征维度激增，组合后的特征值有可能很稀疏。因此，通常需要结合特征含义，组合出有业务含义的特征。

In [5]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# 缺失值处理
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imped_data = imp.fit_transform(german_credit_data[data_utils.numeric_cols])
imped_df = pd.DataFrame(imped_data, columns=data_utils.numeric_cols)
print("缺失值填充结果: \n", imped_df)

缺失值填充结果: 
      duration_in_month  credit_amount  age_in_years  present_residence_since  \
0                  6.0         1169.0          67.0                      4.0   
1                 48.0         5951.0          22.0                      2.0   
2                 12.0         2096.0          49.0                      3.0   
3                 42.0         7882.0          45.0                      4.0   
4                 24.0         4870.0          53.0                      4.0   
..                 ...            ...           ...                      ...   
995               12.0         1736.0          31.0                      4.0   
996               30.0         3857.0          40.0                      4.0   
997               12.0          804.0          38.0                      4.0   
998               45.0         1845.0          23.0                      4.0   
999               45.0         4576.0          27.0                      4.0   

     number_of_existing_cred

In [6]:
# 特征无量纲化
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# max-min标准化
X_MinMaxScaler = MinMaxScaler().fit_transform(german_credit_data[data_utils.numeric_cols])
max_min_df = pd.DataFrame(X_MinMaxScaler, columns=data_utils.numeric_cols)
print("max-min标准化结果: \n", max_min_df)
# z-score标准化
X_StandardScaler = StandardScaler().fit_transform(german_credit_data[data_utils.numeric_cols])
standard_df = pd.DataFrame(X_StandardScaler, columns=data_utils.numeric_cols)
print("z-score标准化结果: \n", standard_df)

max-min标准化结果: 
      duration_in_month  credit_amount  age_in_years  present_residence_since  \
0             0.029412       0.050567      0.857143                 1.000000   
1             0.647059       0.313690      0.053571                 0.333333   
2             0.117647       0.101574      0.535714                 0.666667   
3             0.558824       0.419941      0.464286                 1.000000   
4             0.294118       0.254209      0.607143                 1.000000   
..                 ...            ...           ...                      ...   
995           0.117647       0.081765      0.214286                 1.000000   
996           0.382353       0.198470      0.375000                 1.000000   
997           0.117647       0.030483      0.339286                 1.000000   
998           0.602941       0.087763      0.071429                 1.000000   
999           0.602941       0.238032      0.142857                 1.000000   

     number_of_existing

In [6]:
# 连续特征离散化
from toad.plot import bin_plot

# 利用toad库等频分箱
# 初始化分箱对象
c = toad.transform.Combiner()
c.fit(german_credit_data[data_utils.x_cols],
      y=german_credit_data[data_utils.label], n_bins=6, method='quantile', empty_separate=True)
# 特征age_in_years分箱结果画图
data_binned = c.transform(german_credit_data, labels=True)
bin_plot(data_binned, x='age_in_years', target=data_utils.label)

/opt/conda/lib/python3.7/site-packages/toad/transform.py:271: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(l)
No handles with labels found to put in legend.
No handles with labels found to put in legend.


<AxesSubplot:xlabel='age_in_years', ylabel='prop'>

<Figure size 864x432 with 2 Axes>

In [7]:
# 类别特征数值化
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

def label_encode(x):
    """
    将原始分类变量用标签编码
    :param str x: 需要编码的原始变量
    :returns: x_encoded 标签编码后的变量
    """
    le = LabelEncoder()
    x_encoded = le.fit_transform(x.astype(str))
    class_ = le.classes_
    return class_, pd.DataFrame(x_encoded, columns=x.columns)

def ordinal_encode(x):
    """
    将原始分类变量用数字编码
    :param str x: 需要编码的原始变量，shape为[m,n]
    :returns: x_encoded 数字编码后的变量
    """
    enc = OrdinalEncoder()
    x_encoded = enc.fit_transform(x.astype(str))
    return pd.DataFrame(x_encoded).values

In [9]:
# 以特征purpose为例，进行类别编码
class_, label_encode_x = label_encode(german_credit_data[['purpose']])
print("特征'purpose'的类别编码结果: \n", label_encode_x)
print("特征'purpose'编码顺序为: \n", class_)
# 以特征purpose、credit.history为例，进行类别编码
ordinal_encode_x = ordinal_encode(german_credit_data[['purpose', 'credit_history']])
print("特征'purpose'和'credit_history'的类别编码结果: \n", ordinal_encode_x)

特征'purpose'的类别编码结果: 
      purpose
0          7
1          7
2          4
3          5
4          1
..       ...
995        5
996        2
997        7
998        7
999        2

[1000 rows x 1 columns]
特征'purpose'编码顺序为: 
 ['business' 'car (new)' 'car (used)' 'domestic appliances' 'education'
 'furniture/equipment' 'others' 'radio/television' 'repairs' 'retraining']
特征'purpose'和'credit_history'的类别编码结果: 
 [[7. 1.]
 [7. 3.]
 [4. 1.]
 ...
 [7. 3.]
 [7. 3.]
 [2. 1.]]


/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [9]:
# category_encoders库是专门处理类别特征的
from category_encoders.ordinal import OrdinalEncoder

# 初始化OrdinalEncoder类
encoder = OrdinalEncoder(cols=['purpose', 'personal_status_and_sex'],
                         handle_unknown='value',
                         handle_missing='value')
# 将 handle_unknown设为"value"，即测试集中的未知特征值将被标记为-1
# 将 handle_missing设为"value"，即测试集中的缺失值将被标记为-2
# 当设为"error"，即报错；当设为"return_nan"，即未知值/缺失值被标记为nan
result = encoder.fit_transform(german_credit_data)
category_mapping = encoder.category_mapping
print("类别编码结果: \n", result)
print("类别编码映射关系: \n", category_mapping)


类别编码结果: 
     status_of_existing_checking_account  duration_in_month  \
0                            ... < 0 DM                  6   
1                     0 <= ... < 200 DM                 48   
2                   no checking account                 12   
3                            ... < 0 DM                 42   
4                            ... < 0 DM                 24   
..                                  ...                ...   
995                 no checking account                 12   
996                          ... < 0 DM                 30   
997                 no checking account                 12   
998                          ... < 0 DM                 45   
999                   0 <= ... < 200 DM                 45   

                                        credit_history  purpose  \
0    critical account/ other credits existing (not ...        1   
1             existing credits paid back duly till now        1   
2    critical account/ other credits existin

In [10]:
# 独热编码OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode(x):
    """
    将原始类别变量进行one-hot编码
    :param str x: 需要编码的原始变量
    :returns: x_oht one-hot编码后的变量
    """
    # 首先将类别值进行数值化
    re = OrdinalEncoder()
    x_encoded = re.fit_transform(x.astype(str))
    x_encoded = pd.DataFrame(x_encoded).values
    # 在对数值化后的类别变量进行one-hot编码
    ohe = OneHotEncoder(handle_unknown='ignore')
    x_oht = ohe.fit_transform(x_encoded).toarray()
    return x_oht

In [11]:
# 以特征purpose为例，进行one-hot编码
label_encode_x = one_hot_encode(german_credit_data[['purpose']])
label_encode_df = pd.DataFrame(label_encode_x)
print("特征purpose的one-hot编码结果: \n", label_encode_df)

特征purpose的one-hot编码结果: 
        0    1    2    3    4    5    6    7    8    9
0    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2    0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3    0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4    0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
995  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
996  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
997  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
998  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
999  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

[1000 rows x 10 columns]


In [12]:
# 除了sklearn库以外，还可以使用category_encoders库中的OneHotEncoder类
# 功能更加强大，不但支持直接处理字符串特征值，而且支持对缺失值和未知类别的处理
from category_encoders.one_hot import OneHotEncoder

# 加载数据
german_credit_data = data_utils.get_data()
# 初始化OneHotEncoder类
encoder = OneHotEncoder(cols=['purpose', 'personal_status_and_sex'],
                        handle_unknown='indicator',
                        handle_missing='indicator',
                        use_cat_names=True)
# 转换数据集
result = encoder.fit_transform(german_credit_data)
print("one-hot编码结果: \n", result)

one-hot编码结果: 
     status_of_existing_checking_account  duration_in_month  \
0                            ... < 0 DM                  6   
1                     0 <= ... < 200 DM                 48   
2                   no checking account                 12   
3                            ... < 0 DM                 42   
4                            ... < 0 DM                 24   
..                                  ...                ...   
995                 no checking account                 12   
996                          ... < 0 DM                 30   
997                 no checking account                 12   
998                          ... < 0 DM                 45   
999                   0 <= ... < 200 DM                 45   

                                        credit_history  \
0    critical account/ other credits existing (not ...   
1             existing credits paid back duly till now   
2    critical account/ other credits existing (not ...   
3       

In [13]:
# 目标编码TargetEncoder
from category_encoders.target_encoder import TargetEncoder

y = german_credit_data['creditability']
x = german_credit_data[['purpose', 'personal_status_and_sex']]
# 目标编码
enc = TargetEncoder(cols=x.columns)
result = enc.fit_transform(x, y)
print("目标编码结果: \n", result)

目标编码结果: 
       purpose  personal_status_and_sex
0    0.221429                 0.242846
1    0.221429                 0.242846
2    0.433360                 0.242846
3    0.320442                 0.242846
4    0.380342                 0.242846
..        ...                      ...
995  0.320442                 0.336929
996  0.165082                 0.336929
997  0.221429                 0.336929
998  0.221429                 0.336929
999  0.165082                 0.336929

[1000 rows x 2 columns]


In [14]:
# 证据权重WOE编码
from category_encoders.woe import WOEEncoder

y = german_credit_data['creditability']
x = german_credit_data[['purpose', 'personal_status_and_sex']]

# WOE编码
encoder = WOEEncoder(cols=x.columns)
result = encoder.fit_transform(x, y)
print("WOE编码结果: \n", result)

WOE编码结果: 
       purpose  personal_status_and_sex
0   -0.402431                -0.255106
1   -0.402431                -0.255106
2    0.611705                -0.255106
3    0.100762                -0.255106
4    0.359709                -0.255106
..        ...                      ...
995  0.100762                 0.182108
996 -0.732030                 0.182108
997 -0.402431                 0.182108
998 -0.402431                 0.182108
999 -0.732030                 0.182108

[1000 rows x 2 columns]


In [ ]:
# 特征交叉组合

#TODO


## 4. 特征选择  

### 特征选择（feature selection）是指选择能够使模型获得最佳性能的特征子集。特征选择的必要性包括：其一，特征集合中的特征并非都对模型有益，如果选取不稳定的特征训练模型，那么模型的稳定性较差；其二，线性模型要求特征之间不能多重共线性，因此需要选择无严重多重共线性的特征建模；其三，选取可能对模型有增益的特征，剔除无用特征，从而降低特征维度，缩短模型训练时间和减少对机器资源的消耗。  

### 特征选择往往需要反复迭代、验证，并且和模型训练过程循环进行，最终训练得到性能优异的模型。  

### 1）基于特征属性选择（缺失率选择法、变异系数选择法、相关性选择法、多重共线性选择法）  
### 2）基于特征效果选择（IV选择、卡方检验、包裹法、嵌入法）  
### 3）基于特征稳定性选择（PSI选择、逾期率变化选择）

In [15]:
# 变异系数选择法
# 变异系数（离散系数）是概率分布离散程度的一个归一化量度，其定义为标准差与均值之比
# 相比方差，变异系数是一个无量纲量，因此，更适用于两组量纲不同或均值不同的数据比较
from scipy.stats import variation

# 导入数值型样例数据
all_x_y = data_utils.get_all_x_y()
x = all_x_y.drop(data_utils.label, axis=1)
# 计算各个特征的变异系数
x_var = variation(x, nan_policy='omit')
result = dict(zip(x.columns ,x_var))
print("变异系数结果: \n", result)

变异系数结果: 
 {'status_of_existing_checking_account': 0.6882533114687077, 'credit_history': 0.47927137155432403, 'savings_account_and_bonds': 1.365519914018727, 'present_employment_since': 0.6634558702659792, 'personal_status_and_sex': 0.7189713146288836, 'other_debtors_or_guarantors': 0.23848946827575135, 'property': 0.7370527091490185, 'other_installment_plans': 0.46404208219431514, 'housing': 0.4957965609313624, 'job': 0.7425464900473785, 'telephone': 1.2145976802021627, 'foreign_worker': 0.1960142830746339, 'purpose': 0.7024955444588786, 'duration_in_month': 0.7130558798689015, 'credit_amount': 0.9338246367916342, 'age_in_years': 0.687161816595272, 'present_residence_since': 0.5979218963009554, 'number_of_existing_credits_at_this_bank': 1.4185886210541587, 'installment_rate_in_percentage_of_disposable_income': 0.5667284222332614, 'number_of_people_being_liable_to_provide_maintenance_for': 2.334868926348073}


In [16]:
# 相关性选择法
# 利用pandas库计算相关系数

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
#x = all_x_y.drop(data_utils.label, axis=1)

# pearson相关系数
pearson_corr = x.corr(method='pearson')
print("pandas库计算 pearson相关系数: \n", pearson_corr)
# spearman相关系数
spearman_corr = x.corr(method='spearman')  
print("pandas库计算 spearman相关系数: \n", spearman_corr)
# kendall相关系数
kendall_corr = x.corr(method='kendall')  
print("pandas库计算 kendall相关系数: \n", kendall_corr)

pandas库计算 pearson相关系数: 
                                                     status_of_existing_checking_account  \
status_of_existing_checking_account                                            1.000000   
credit_history                                                                -0.068560   
savings_account_and_bonds                                                      0.238185   
present_employment_since                                                       0.016692   
personal_status_and_sex                                                       -0.006925   
other_debtors_or_guarantors                                                    0.087751   
property                                                                      -0.001884   
other_installment_plans                                                        0.015542   
housing                                                                       -0.016256   
job                                                              

In [17]:
# 利用SciPy库计算Pearson相关系数和p值，p值能够反映两个特征的显著水平
# 如果不显著，那么相关系数再高，也无参考价值，可能是由偶然因素引起的
# 通常情况下，当p值小于0.05时，可以认为统计量是显著的
from scipy.stats import pearsonr

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
#x = all_x_y.drop(data_utils.label, axis=1)
x1, x2 = x.loc[:, 'age_in_years'], x.loc[:, 'credit_history',]
r, p_value = pearsonr(x1, x2)
print("scipy库计算 特征'age_in_years'和'credit_history'的pearson相关系数 \n", 
    "pearson相关系数: %s, \n" % r, "p_value: %s" % p_value)

scipy库计算 特征'age_in_years'和'credit_history'的pearson相关系数 
 pearson相关系数: -0.15726149553005797, 
 p_value: 5.794147610911892e-07


In [18]:
# 多重共线性选择法
# 方差膨胀系数VIF是一种衡量共线性程度的常用指标，表示回归系数估计量的方差与假设特征间不线性相关时的方差的比值
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
#x = all_x_y.drop(data_utils.label, axis=1)
vif = [variance_inflation_factor(x.values, ix) for ix in range(x.shape[1])]
print("各特征的vif值计算结果: \n", dict(zip(x.columns, vif)))

# 筛选阈值小于10的特征
selected_cols = x.iloc[:, [f < 10 for f in vif]].columns.tolist()
print("设置vif阈值为10, 筛选得到%s个特征: \n" % len(selected_cols), selected_cols)

各特征的vif值计算结果: 
 {'status_of_existing_checking_account': 3.3855245279124184, 'credit_history': 5.946875559126981, 'savings_account_and_bonds': 1.6913441790841452, 'present_employment_since': 3.2163691264728365, 'personal_status_and_sex': 2.9005718610019535, 'other_debtors_or_guarantors': 16.048334035069896, 'property': 3.109607867242475, 'other_installment_plans': 5.537475799354373, 'housing': 5.546292850678612, 'job': 3.33617284726315, 'telephone': 2.1188507642338843, 'foreign_worker': 22.97118539735503, 'purpose': 3.1762764899150255, 'duration_in_month': 5.651956673719252, 'credit_amount': 4.5773830165809155, 'age_in_years': 3.7860509603745838, 'present_residence_since': 4.2683898280429275, 'number_of_existing_credits_at_this_bank': 1.7904787406064542, 'installment_rate_in_percentage_of_disposable_income': 5.019114067135304, 'number_of_people_being_liable_to_provide_maintenance_for': 1.259420463744714}
设置vif阈值为10, 筛选得到18个特征: 
 ['status_of_existing_checking_account', 'credit_history', 

In [19]:
# IV（Infomation Value，信息价值）选择，是衡量特征预测能力的关键指标
# WOE描述了特征和目标变量之间的关系，IV用于衡量这个关系的强弱程度
# 一般认为贷前特征IV小于0.02表示该特征几乎无预测能力；[0.02,01)区间代表特征预测能力弱；[0.1,0.3)区间代表特征预测能力中等；
# [0.3,05)代表特征预测能力强；当IV大于等于0.5时，需要确认特征逻辑是否正常，以及是否存在“穿越”情况

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()

# 利用toad库quality()方法计算IV
var_iv = toad.quality(all_x_y,
                      target='creditability',
                      method='quantile',
                      n_bins=6,
                      iv_only=True)

selected_cols = var_iv[var_iv.iv > 0.1].index.tolist()
print("各特征的iv值计算结果: \n", var_iv)
print("设置iv阈值为0.1, 筛选得到%s个特征: \n" % len(selected_cols), selected_cols)

各特征的iv值计算结果: 
                                                           iv  gini  entropy  \
status_of_existing_checking_account                 0.666012   NaN      NaN   
credit_history                                      0.293234   NaN      NaN   
duration_in_month                                   0.265263   NaN      NaN   
savings_account_and_bonds                           0.196010   NaN      NaN   
purpose                                             0.169195   NaN      NaN   
property                                            0.112638   NaN      NaN   
present_employment_since                            0.086434   NaN      NaN   
housing                                             0.083293   NaN      NaN   
credit_amount                                       0.082114   NaN      NaN   
age_in_years                                        0.065025   NaN      NaN   
other_installment_plans                             0.057615   NaN      NaN   
foreign_worker                       

In [20]:
# 卡方检验，是一种以卡方分布为基础的检验方法，主要用于类别变量
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
y = all_x_y.pop(data_utils.label)

# 选择K个最好的特征，返回选择特征后的数据
fs_chi = SelectKBest(chi2, k=5)
fs_chi.fit(all_x_y, y)
x_new = fs_chi.transform(all_x_y)

selected_cols = all_x_y.columns[fs_chi.get_support()].tolist()
print("卡方检验筛选得到%s个特征: \n" % len(selected_cols), selected_cols)

卡方检验筛选得到5个特征: 
 ['status_of_existing_checking_account', 'savings_account_and_bonds', 'purpose', 'duration_in_month', 'credit_amount']


In [21]:
# 逐步stepwise回归，是一种筛选并剔除引起多重共线性变量的方法，在Logistic回归中应用广泛

# 导入数值型样例数据
all_x_y = data_utils.get_all_x_y()

final_data = toad.selection.stepwise(all_x_y,
                                     target=data_utils.label,
                                     estimator='lr',
                                     direction='both',
                                     criterion='aic',
                                     return_drop=False)
selected_cols = final_data.columns
print("通过stepwise筛选得到%s个特征: \n" % len(selected_cols), selected_cols)


通过stepwise筛选得到13个特征: 
 Index(['status_of_existing_checking_account', 'credit_history',
       'savings_account_and_bonds', 'present_employment_since',
       'other_installment_plans', 'telephone', 'foreign_worker', 'purpose',
       'duration_in_month', 'credit_amount', 'age_in_years',
       'installment_rate_in_percentage_of_disposable_income', 'creditability'],
      dtype='object')


In [22]:
# 递归特征消除RFE，是使用一个基模型进行多轮训练，每轮训练后消除若干重要性低的特征，再基于新特征集进行下一轮训练

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# 导入数值型样例数据
all_x_y = data_utils.get_all_x_y()
y = all_x_y.pop(data_utils.label)
x = all_x_y
# 递归特征消除法，返回特征选择后的数据
# 参数estimator为基模型
# 参数n_features_to_select为选择的特征个数
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=10)
x_new = rfe.fit_transform(x, y)

selected_cols = x.columns[rfe.get_support()].tolist()
print("通过递归特征消除法筛选得到%s个特征: \n" % len(selected_cols), selected_cols)

通过递归特征消除法筛选得到10个特征: 
 ['status_of_existing_checking_account', 'credit_history', 'savings_account_and_bonds', 'other_installment_plans', 'foreign_worker', 'purpose', 'duration_in_month', 'credit_amount', 'age_in_years', 'installment_rate_in_percentage_of_disposable_income']


In [23]:
# 基于L1范数的特征选择
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# 导入数值型样例数据
all_x_y = data_utils.get_all_x_y()
y = all_x_y.pop(data_utils.label)
x = all_x_y

# 带L1惩罚项的逻辑回归作为基模型的特征选择
LR = LogisticRegression(penalty='l1', C=0.1, solver='liblinear')
sf = SelectFromModel(LR)
x_new = sf.fit_transform(x, y)

selected_cols = x.columns[sf.get_support()].tolist()
print("基于L1范数筛选得到%s个特征: \n" % len(selected_cols), selected_cols)

基于L1范数筛选得到6个特征: 
 ['status_of_existing_checking_account', 'savings_account_and_bonds', 'present_employment_since', 'purpose', 'duration_in_month', 'age_in_years']


In [24]:
# 基于树模型GBDT的特征选择
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier

# 导入数值型样例数据
all_x_y = data_utils.get_all_x_y()
y = all_x_y.pop(data_utils.label)
x = all_x_y

# GBDT作为基模型的特征选择
sf = SelectFromModel(GradientBoostingClassifier())
x_new = sf.fit_transform(x, y)

selected_cols = x.columns[sf.get_support()].tolist()
print("基于树模型筛选得到%s个特征: \n" % len(selected_cols), selected_cols)

基于树模型筛选得到7个特征: 
 ['status_of_existing_checking_account', 'credit_history', 'savings_account_and_bonds', 'purpose', 'duration_in_month', 'credit_amount', 'age_in_years']


### 注意：除使用单一模型的特征重要度进行选择之外，常用方法是依据多个模型的特征重要度综合排名进行特征选择，如先分别采样随机森林和XGBoost训练模型，再根据两个模型的特征重要度加权得到综合排序，最终依据综合排序选择特征

In [25]:
# 基于特征的稳定性指标PSI的特征选择

# 加载数据
all_x_y = data_utils.get_all_x_y()

# 定义分箱方法
Combiner = toad.transform.Combiner()
Combiner.fit(all_x_y,
             y=data_utils.label,
             n_bins=6,
             method='quantile',
             empty_separate=True)
# 计算psi
var_psi = toad.metrics.PSI(all_x_y.iloc[:500, :],
                           all_x_y.iloc[500:, :],
                           combiner=Combiner)
var_psi_df = var_psi.to_frame(name='psi')

selected_cols = var_psi[var_psi_df.psi < 0.1].index.tolist()
print("各特征的psi值计算结果: \n", var_psi_df)
print("设置psi阈值为0.1, 筛选得到%s个特征: \n" % len(selected_cols), selected_cols)


各特征的psi值计算结果: 
                                                          psi
status_of_existing_checking_account                 0.010177
credit_history                                      0.025913
savings_account_and_bonds                           0.001666
present_employment_since                            0.005920
personal_status_and_sex                             0.624539
other_debtors_or_guarantors                         0.000047
property                                            0.017745
other_installment_plans                             0.002711
housing                                             0.002367
job                                                 0.008619
telephone                                           0.002393
foreign_worker                                      0.005565
purpose                                             0.020281
duration_in_month                                   0.034919
credit_amount                                       0.010007
age_in_y

In [26]:
# 基于逾期率变化的特征选择

# 导入添加month列的数据
german_credit_data = data_utils.get_data()

x = german_credit_data[data_utils.x_cols]
y = german_credit_data[data_utils.label]

# 分箱
Combiner = toad.transform.Combiner()
x_cat = Combiner.fit_transform(x, y, n_bins=6, method='quantile', empty_separate=True)

# 合并标签和month
x_cat_with_month = x_cat.merge(german_credit_data[['month', 'creditability']], left_index=True, right_index=True)

# 单个特征对比逾期率
feature_col = 'age_in_years'
x_cat_one = x_cat_with_month[[feature_col, 'month', 'creditability']]
feature_var = x_cat_one.pivot_table(index=feature_col,
                                columns='month',
                                values='creditability',
                                aggfunc=['mean'])
print("特征'age_in_years'的按月分箱逾期率统计结果: \n", feature_var)


# 计算特征按月逾期率波动值
def variation_by_month(df, time_col, columns, label_col):
    variation_dict = {}
    for col in columns:
        feature_v = df.pivot_table(
            index=col, columns=time_col, values=label_col, aggfunc=['mean'])
        variation_dict[col] = feature_v.rank().std(axis=1).mean()

    return pd.DataFrame([variation_dict], index=['variation']).T


var_badrate = variation_by_month(x_cat_with_month, 'month', data_utils.x_cols, 'creditability')
print("各特征按月逾期率的标准差: \n", var_badrate)

selected_cols = var_badrate[var_badrate['variation'] < 0.8].index.tolist()
print("设置标准差阈值为0.8, 筛选得到%s个特征: \n" % len(selected_cols), selected_cols)


特征'age_in_years'的按月分箱逾期率统计结果: 
                   mean                                        
month          2020-01   2020-02   2020-03   2020-04   2020-05
age_in_years                                                  
0             0.407407  0.304348  0.352941  0.439024  0.541667
1             0.393939  0.314286  0.407407  0.210526  0.250000
2             0.394737  0.218750  0.307692  0.350000  0.302326
3             0.250000  0.232558  0.285714  0.216216  0.300000
4             0.166667  0.266667  0.280000  0.311111  0.250000
5             0.297297  0.352941  0.218750  0.230769  0.153846
各特征按月逾期率的标准差: 
                                                     variation
duration_in_month                                    0.837724
credit_amount                                        1.378574
age_in_years                                         1.445965
present_residence_since                              1.189319
number_of_existing_credits_at_this_bank              0.447214
installment_r

/opt/conda/lib/python3.7/site-packages/toad/transform.py:271: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(l)


## 5. 特征提取  

### 特征提取（feature extraction）是指从原有较多的特征中计算出较少的新特征，用新特征替换原有特征，达到降维的目的，即通过从样本中学习一个映射函数 *f*，将原特征矩阵 *X1* 映射到新的特征矩阵 *X2*，其中 *X2* 的维度低于 *X1*。  

### 不同于特征选择的是，特征提取是通过属性间的关系，如组合不同属性得到新属性，这样就改变了原有的特征空间。而特征选择是从原始特征集合中选出子集，这是一种包含关系，没有更改原始特征空间。特征提取会或多或少地损失一部分特征信息。  

### 1）线性特征提取（PCA、LDA）  
### 2）非线性特征提取（LLE、MDS）

In [27]:
# PCA将高维的特征向量合并为低维的特征向量，是一种无监督的特征提取方法
from sklearn.decomposition import PCA

# 导入数值型样例数据
all_x_y = data_utils.get_all_x_y()
x = all_x_y.drop(data_utils.label, axis=1)

pca = PCA(n_components=0.9)
x_new = pca.fit_transform(x)
x_new_df = pd.DataFrame(x_new)
print("利用sklearn进行PCA特征提取, 保留90%信息后结果: \n", x_new_df)

利用sklearn进行PCA特征提取, 保留90%信息后结果: 
            0         1         2         3         4         5         6   \
0    0.668144  0.247366 -0.167536  0.487377 -0.624058  0.590125  0.161462   
1   -0.399327  0.365602 -0.036677 -0.310236  0.188022 -0.250206 -0.274869   
2   -0.460871  0.307393  0.254944  0.705694  0.485358 -0.631305  0.031393   
3   -0.406718  0.226530 -0.557889  0.682061  0.388980  0.176492  0.271157   
4   -0.287479  0.223968 -0.641140  1.186046  0.041876 -0.363786  0.139752   
..        ...       ...       ...       ...       ...       ...       ...   
995 -0.452867 -0.006116  0.243526  0.153927 -0.164695 -0.245425 -0.562441   
996  0.528161 -0.135177 -0.812950 -0.169609 -0.284602  0.511227  0.105754   
997 -0.305685 -0.024248  0.205467 -0.123406 -0.554920 -0.151249 -0.316047   
998  0.477465 -0.031619 -0.868358  0.211375 -0.560959 -0.083677 -0.158878   
999 -0.117698 -0.093517  0.259515  0.183946 -0.135540  0.245897 -0.276917   

           7         8         9         

In [28]:
# LDA是一种基于分类模型进行特征属性合并的操作，是一种有监督的特征提取方法
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
#x = all_x_y.drop(data_utils.label, axis=1)
#y = all_x_y[data_utils.label]

lda = LinearDiscriminantAnalysis(n_components=1)
x_new = lda.fit_transform(x, y)
x_new_df = pd.DataFrame(x_new)
print("利用sklearn进行LDA特征提取结果: \n", x_new_df)

利用sklearn进行LDA特征提取结果: 
             0
0   -0.811651
1    1.033838
2   -1.438663
3    1.545152
4    1.417457
..        ...
995 -0.764533
996  1.589010
997 -0.716811
998  1.650618
999  0.411717

[1000 rows x 1 columns]


In [29]:
# LLE是一种基于“流形学习”的方法，能够使特征提取后的数据较好地保持原有流形结构
from sklearn.manifold import LocallyLinearEmbedding

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
#x = all_x_y.drop(data_utils.label, axis=1)

lle = LocallyLinearEmbedding(n_neighbors=5, n_components=10)
x_new = lle.fit_transform(x)
x_new_df = pd.DataFrame(x_new)
print("利用sklearn进行LLE特征提取结果: \n", x_new_df)

利用sklearn进行LLE特征提取结果: 
             0         1         2         3         4         5         6  \
0   -0.020305  0.055740  0.004281 -0.009044  0.006460 -0.012759 -0.003094   
1    0.025944  0.002840  0.076652 -0.032942  0.056135 -0.002621  0.010546   
2    0.025693  0.001902  0.024466  0.071755 -0.013074 -0.012395 -0.002964   
3    0.025683  0.001712  0.031975  0.029800  0.016058 -0.007725 -0.001374   
4    0.025537  0.001931  0.026930  0.063353 -0.004551 -0.003255 -0.001462   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.026142 -0.000201 -0.006878 -0.012808 -0.003007 -0.001415 -0.008683   
996 -0.037399 -0.014598 -0.001767 -0.002825  0.000188 -0.008735 -0.001364   
997  0.026149  0.000372  0.008647 -0.016476  0.006198 -0.001462 -0.018741   
998 -0.041306 -0.014037 -0.000939 -0.002173 -0.000371 -0.011253 -0.000483   
999  0.026347 -0.001203 -0.034766 -0.006164 -0.002894 -0.001269  0.029846   

            7         8         9  
0   -0.003019 -

In [30]:
# MDS是将高维空间中的样本点投影到低维空间中，让样本彼此之间的距离尽可能不变
from sklearn.manifold import MDS

# 导入数值型样例数据
#all_x_y = data_utils.get_all_x_y()
#x = all_x_y.drop(data_utils.label, axis=1)

mds = MDS(n_components=10)
x_new = mds.fit_transform(x)
x_new_df = pd.DataFrame(x_new)
print("利用sklearn进行MDS特征提取结果: \n", x_new_df)

利用sklearn进行MDS特征提取结果: 
             0         1         2         3         4         5         6  \
0   -0.063017  0.179366  0.003586  0.085858  0.440596 -0.766275 -0.939513   
1   -0.068954 -0.536181 -0.033155  0.405570 -0.347190  0.150980  0.275525   
2   -0.341509  0.161344  0.380788 -0.514796 -0.251431  0.311106  0.843898   
3   -0.303862 -0.032223 -0.477742 -0.731142  0.804963 -0.233156  0.795239   
4   -0.514585 -0.024780 -0.743554 -0.387596  0.308766 -0.236328  0.441279   
..        ...       ...       ...       ...       ...       ...       ...   
995 -0.463773 -0.540679  0.604554 -0.158095 -0.244723  0.118035  0.262113   
996  0.358472 -0.140490 -0.566637 -0.465408  0.515999 -0.283529 -0.698974   
997 -0.421897 -0.499534  0.287899 -0.062857 -0.159667  0.413332 -0.362481   
998  0.423967 -0.642201 -0.414688  0.178004  0.166478 -0.613619 -0.638501   
999 -0.385022 -0.336766 -0.119083 -0.330462  0.117684 -0.241830  0.446720   

            7         8         9  
0   -0.743548 -

## 6. 模型训练  
机器学习的模型训练，本质上是参数优化过程。参数分为两种，一种是模型参数（parameter），另一种是超参数（hyperparameter）。模型参数是可以通过数据估计得到的，如线性回归模型中的回归系数。超参数是用来定义模型结构或优化策略的，通常需要在模型训练前根据经验给定，如正则化系数、学习率。给定不同的超参数组合，往往训练得到的模型参数是不同的，模型效果也有差异。  

**模型调参**是寻找最优超参数组合的过程。常见的模型调参方法有：**网格搜索**（grid search）、**随机搜索**（random search）、**贝叶斯优化**（bayesian optimization）  

**模型评估**，在模型调参过程中，如何评估模型的真实效果呢？通常从样本中随机抽取一定比例的样本作为验证集，并用验证集上的模型效果作为模型调参评判的依据。如果训练集和验证集的划分不合适，最终模型的效果也会大打折扣。另外，这种方法只使用了部分样本建模，没有用到全部数据信息。这时可以采用交叉验证（Cross Validation）方式评估模型效果，如K折（K-fold）交叉验证。  

**最终模型**，在得到最优参数组合之后，可以在完整的样本（训练样本+验证样本）上再次训练，得到最终模型，该模型效果将在测试样本或OOT样本上进一步进行评估。  

### 1）LR算法  
### 2）支持向量机SVM  
### 3）决策树  
### 4）集成学习  
### 5）深度学习

## 7. 概率转化  
### 在信贷风控场景中，需要将模型输出的客户逾期概率转化为评分，通过分数量化客户的风险等级

In [31]:
def p_to_score(p, pdo, base, odds):
    """ 
    逾期概率转换分数 
    :param p: 逾期概率 
    :param pdo: points double odds. default = 60 
    :param base: base points. default = 600 
    :param odds: odds. default = 1.0/15.0 
    :returns: 模型分数 
    """
    B = pdo / np.log(2)
    A = base + B * np.log(odds)
    score = A - B * np.log(p / (1 - p))
    return round(score, 0)

In [32]:
pros = pd.Series(np.random.rand(100))
pros_score = p_to_score(pros, pdo=60.0, base=600, odds=1.0 / 15.0)
print("随机产生100个概率并转化为score结果: \n", dict(zip(pros, pros_score)))

随机产生100个概率并转化为score结果: 
 {0.6102427946018817: 327.0, 0.08231964289808436: 574.0, 0.3517316151991714: 419.0, 0.05924666108681864: 605.0, 0.5511942059403694: 348.0, 0.030224544223225736: 666.0, 0.646718491350247: 313.0, 0.7698108725295926: 261.0, 0.249038332134491: 461.0, 0.45795312161913126: 380.0, 0.30718887195024713: 436.0, 0.13416449346124337: 527.0, 0.6835869595498852: 299.0, 0.7863112951916512: 253.0, 0.2433086389020468: 464.0, 0.8844310747373691: 189.0, 0.20910137303141652: 481.0, 0.8539159509381035: 213.0, 0.10262518923085906: 553.0, 0.6598146929623041: 308.0, 0.42566865526802855: 392.0, 0.0236685429958714: 688.0, 0.9405662994891044: 127.0, 0.20048745265166457: 485.0, 0.8261477633042558: 231.0, 0.7707418397236426: 261.0, 0.8934020834945413: 182.0, 0.9543778339569833: 102.0, 0.1981296794636631: 487.0, 0.6411370258026878: 315.0, 0.32961094914560685: 427.0, 0.7601692165427684: 266.0, 0.14600726239894046: 518.0, 0.3187517484183914: 431.0, 0.18181078736955836: 496.0, 0.922784344509222

## 8. 模型效果评估  

### 根据模型对样本的预测分数和样本的真实标签，可以通过不同角度的指标评估模型的效果。  

### 对于分类任务，常用的评价指标有准确率、精确率、召回率、F1、AUC和KS等。  

### 对于回归任务，常用的评价指标有RMSE（平方根误差）、MAE（平均绝对误差）、MSE（平均平方误差）和coefficient of determination（决定系数）等。

In [33]:

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import validation_curve

X, y = load_digits(return_X_y=True)

param_range = np.logspace(-6, -1, 5)
train_scores, test_scores = validation_curve(
    SVC(), X, y, param_name="gamma", param_range=param_range,
    scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with SVM")
plt.xlabel(r"$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

<Figure size 432x288 with 1 Axes>

## 9. 模型部署及上线验证  

### 模型训练通常在本地环境中进行，训练完成后需要选择最优模型部署到生产环境。

## 10. 模型监控及迭代优化  

### 模型上线后，客群的变化可能导致模型效果衰减；或者，随着数据维度的增加，模型效果进一步提升；亦或，由于业务调整，某些数据维度无法再使用。  

### 模型迭代优化可以使得模型在近期样本上表现得更好，可以从模型融合、建模时效和拒绝推断3个角度进行。